### Load Data

In [1]:
!pip install cassis

ERROR: Could not find a version that satisfies the requirement cassis (from versions: none)
ERROR: No matching distribution found for cassis

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from cassis import *
import os
test_set_ids = ['lusa_97',
 'lusa_4',
 'lusa_67',
 'lusa_20',
 'lusa_83',
 'lusa_104',
 'lusa_80',
 'lusa_79',
 'lusa_34',
 'lusa_47',
 'lusa_30',
 'lusa_96',
 'lusa_11',
 'lusa_112',
 'lusa_100',
 'lusa_77',
 'lusa_38',
 'lusa_86',
 'lusa_60']

#move test_files to evaluation folder

def load_aligned_files(docs_path):
	aligner_files_names = []
	aligner_files = []
	aligner_files_dirs = []
	for file_ in os.listdir(docs_path):
		file__ = file_.strip(".zip")
		if file__ in test_set_ids:
			for file in os.listdir(os.path.join(docs_path, file_)):
				if file == "INITIAL_CAS.json":
					aligner_files_dirs.append((os.path.join(docs_path, file_ +"/"+ file), file__))
					with open(os.path.join(docs_path, file_ +"/"+ file), "r") as f:
						cas = load_cas_from_json(f)
						aligner_files.append(cas)
						aligner_files_names.append(file__)

	return aligner_files
Mtrans_path = "./no_Mtrans/annotation"
lemma_path = "./no_lemma/annotation"
fuzzy_path = "./no_fuzzy/annotation"
wAligner_path = "./no_wAligner/annotation"
overall_path = "../validation/lingualigner/"

mtrans_files = load_aligned_files(Mtrans_path)
lemma_files = load_aligned_files(lemma_path)
fuzzy_files = load_aligned_files(fuzzy_path)
wAligner_files = load_aligned_files(wAligner_path)
overall_files = load_aligned_files(overall_path)

### Load Reviewed Data

In [ ]:
nana_files = []
nana_path = "../validation/nana/annotation"
nana_files_names = []
for file_ in os.listdir(nana_path):
	file__ = file_.strip(".zip")
	if file__ in test_set_ids:
		for file in os.listdir(os.path.join(nana_path, file_)):
			if file == "storysense_anotador.json":
				with open(os.path.join(nana_path, file_ +"/"+ file), "r") as f:
					cas = load_cas_from_json(f)
					nana_files.append(cas)
					nana_files_names.append(file__)


In [76]:
mtrans_spans = [cas.select("custom.Span") for cas in mtrans_files]
lemma_spans = [cas.select("custom.Span") for cas in lemma_files]
fuzzy_spans = [cas.select("custom.Span") for cas in fuzzy_files]
wAligner_spans = [cas.select("custom.Span") for cas in wAligner_files]
overall_spans = [cas.select("custom.Span") for cas in overall_files]
nana_spans = [cas.select("custom.Span") for cas in nana_files]

#sample = aligner_spans[0][0]
#sample

In [ ]:

mtrans_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in mtrans_spans]
lemma_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in lemma_spans]
fuzzy_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in fuzzy_spans]
wAligner_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in wAligner_spans]
overall_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in overall_spans]
nana_spans_dicts = [dict((span.xmiID, span) for span in spans) for spans in nana_spans]


In [82]:
#get text for each  nana and aligner spans (with same id) and calculate f1 score on token level
def calculate_exact_match(aligner_spans_dicts, nana_spans_dicts):
	errors = 0
	corrects = 0
	total = 0
	for aligner_dict, nana_dict in zip(aligner_spans_dicts, nana_spans_dicts):
		for xmi_id in aligner_dict.keys():
			if xmi_id in nana_dict:
				aligner_span = aligner_dict[xmi_id]
				nana_span = nana_dict[xmi_id]
				if nana_span.Aligned == True:
					if nana_span.begin == aligner_span.begin and nana_span.end == aligner_span.end:
						corrects += 1
					else:
						errors += 1
				total += 1
	return errors, corrects, total

mtrans_errors, mtrans_corrects, mtrans_total = calculate_exact_match(mtrans_spans_dicts, nana_spans_dicts)
lemma_errors, lemma_corrects, lemma_total = calculate_exact_match(lemma_spans_dicts, nana_spans_dicts)
fuzzy_errors, fuzzy_corrects, fuzzy_total = calculate_exact_match(fuzzy_spans_dicts, nana_spans_dicts)
wAligner_errors, wAligner_corrects, wAligner_total = calculate_exact_match(wAligner_spans_dicts, nana_spans_dicts)
overall_errors, overall_corrects, overall_total = calculate_exact_match(overall_spans_dicts, nana_spans_dicts)

#print exact match results

mtrans_exact_match_percentage = round((mtrans_corrects / mtrans_total) * 100, 2)
print("Exact match removing M_Trans:", mtrans_exact_match_percentage)
lemma_exact_match_percentage = round((lemma_corrects / lemma_total) * 100, 2)
print("Exact match removing Lemma:", lemma_exact_match_percentage)
fuzzy_exact_match_percentage = round((fuzzy_corrects / fuzzy_total) * 100, 2)
print("Exact match removing Fuzzy:", fuzzy_exact_match_percentage)
wAligner_exact_match_percentage = round((wAligner_corrects / wAligner_total) * 100, 2)
print("Exact match removing Word_aligner:", wAligner_exact_match_percentage)
overall_exact_match_percentage = round((overall_corrects / overall_total) * 100, 2)
print("Exact match removing Overall:", overall_exact_match_percentage)


Exact match removing M_Trans: 60.43
Exact match removing Lemma: 75.72
Exact match removing Fuzzy: 72.12
Exact match removing Word_aligner: 68.6
Exact match removing Overall: 75.96
